In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('/Users/Aaron_hill/repos/visualizedata/ml/week01/metadata/definitions.csv')

# create a boolean indicator =True if the definition is of "machine learning"
data['ml'] = data['term'] == 'machine learning'

print(data.dtypes)
print(data.head(10))

term          object
definition    object
ml              bool
dtype: object
               term                                         definition    ml
0  machine learning  Machine learning is the development of functio...  True
1  machine learning  it is training an algorithm on how to make pre...  True
2  machine learning  Computers making decisions and learning throug...  True
3  machine learning  Creating a model to teach an algorithm to reco...  True
4  machine learning  Applying a model to a large amount of data to ...  True
5  machine learning  A subset/practice of Artificial Intelligence w...  True
6  machine learning   let machine read a lot of data, and conclude ...  True
7  machine learning  machine process of aggregating information in ...  True
8  machine learning  Automatic or semi-automatic updating of equati...  True
9  machine learning  Machine Learning is a method of computation th...  True


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

L = data["ml"] # labels
corpus = data['definition'] # corpus of reviews in words
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.shape)

(107, 494)


## Extract polynomial features

In [4]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
Xp = poly.fit_transform(X)
Xp.shape

(107, 122760)

## Polynomial features with the kernel trick

In [5]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
rbf_feature = RBFSampler(gamma=1, random_state=1, n_components=123000)
X_features = rbf_feature.fit_transform(X)
# fit SGDClassifier
clf = SGDClassifier(max_iter=5)
clf.fit(X_features, L)   


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [6]:
X_features.shape

(107, 123000)

In [7]:
import my_measures

performance_measures = my_measures.BinaryClassificationPerformance(clf.predict(X_features), L, 'clf')
performance_measures.compute_measures()
print(performance_measures.performance_measures)

{'Pos': 35, 'Neg': 72, 'TP': 35, 'TN': 72, 'FP': 0, 'FN': 0, 'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'desc': 'clf'}


In [9]:
# machine learning definitions
# Wikipedia, Expert System, Tech Emergence
ml_defs = ["Machine learning is a field of computer science that gives computers the ability to learn without being explicitly programmed.",
          "Machine learning is an application of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. Machine learning focuses on the development of computer programs that can access data and use it learn for themselves.",
          "Machine Learning is the science of getting computers to learn and act like humans do, and improve their learning over time in autonomous fashion, by feeding them data and information in the form of observations and real-world interactions."]
    
# AI definitions
# Wikipedia, Oxford dictionary
ai_defs = ["Artificial intelligence (AI, also machine intelligence, MI) is intelligence demonstrated by machines, in contrast to the natural intelligence (NI) displayed by humans and other animals.",
          "the theory and development of computer systems able to perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and translation between languages"]

# Definitions of unrelated things: kitten, piano, widget
other_defs = ["A kitten, also known as a kitty or kitty cat, is a juvenile cat.",
             "The piano is an acoustic, stringed musical instrument invented in Italy by Bartolomeo Cristofori around the year 1700 in which the strings are struck by hammers.",
             "a small gadget or mechanical device, especially one whose name is unknown or unspecified"]
    
def get_prediction(definition):
    text_x = vectorizer.transform([definition]).toarray()
    rbf_feature = RBFSampler(gamma=1, random_state=1, n_components=123000)
    X_features = rbf_feature.fit_transform(text_x)
    return(clf.predict(X_features))

print("Model predictions for 'machine learning' definitions:")
for mld in ml_defs:
    print(get_prediction(mld))
    
print("* * *")
print("Model predictions for 'AI' definitions:")
for aid in ai_defs:
    print(get_prediction(aid))

print("* * *")
print("Model predictions for other definitions (kitten, piano, widget):")
for otherd in other_defs:
    print(get_prediction(otherd))

Model predictions for 'machine learning' definitions:
[False]
[False]
[False]
* * *
Model predictions for 'AI' definitions:
[False]
[False]
* * *
Model predictions for other definitions (kitten, piano, widget):
[False]
[False]
[False]
